### Load packages and data files from previous notebook

In [179]:
## Imports
from scipy import io
import pandas as pd
import numpy as np
import os
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

In [180]:
## Load Matlab files
file_path = r'C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\interim'
AMO_file, biomass_file, ecosystem_file, landings_file, ocean_temp_file = [fr'{file_path}\AMO_data.csv', \
    fr'{file_path}\biomass_data.csv', fr'{file_path}\ecosystem_data.csv', fr'{file_path}\landings_data.csv', \
    fr'{file_path}\ocean_temp_data.csv']

AMO_df = pd.read_csv(AMO_file).drop(['Unnamed: 0'], axis=1)
biomass_df = pd.read_csv(biomass_file).drop(['Unnamed: 0'], axis=1)
ecosystem_df = pd.read_csv(ecosystem_file).drop(['Unnamed: 0'], axis=1)
landings_df = pd.read_csv(landings_file).drop(['Unnamed: 0'], axis=1)
ocean_temp_df = pd.read_csv(ocean_temp_file).drop(['Unnamed: 0'], axis=1)

### Generate df reports and save to files

In [181]:
def make_html_report (df, name):
    report = df.profile_report(sort=None, html={'style':{'full_width': True}}, progress_bar=True)
    report.to_file(fr"C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\reports\{name}_data_report.html")
    print(f'{name} done!')
'''
make_html_report(AMO_df, 'AMO')
make_html_report(biomass_df, 'biomass')
make_html_report(ecosystem_df, 'ecosystem')
make_html_report(landings_df, 'landings')
make_html_report(ocean_temp_df, 'ocean_temp')
'''

"\nmake_html_report(AMO_df, 'AMO')\nmake_html_report(biomass_df, 'biomass')\nmake_html_report(ecosystem_df, 'ecosystem')\nmake_html_report(landings_df, 'landings')\nmake_html_report(ocean_temp_df, 'ocean_temp')\n"

## Data defining:

### In Biomass:
biomass in tons per square kilometer (t/km^2).
biomass_index in tons.
abundance in number per square kilometer.
abundance_index in number.

### In Landings:
landings_biomass in tons per square kilometer (t/km^2).
landings_biomass_index in tons.
landings_abundance in number per square kilometer.
landings_abundance_index in number.

In [182]:
biomass_spec =  list(set(biomass_df['species'].values))
biomass_loc = list(set(biomass_df['ecosystem'].values))
landings_spec = list(set(landings_df['species'].values))
landings_loc = list(set(landings_df['ecosystem'].values))
biomass_spec.sort()
biomass_loc.sort()
landings_spec.sort()
landings_loc.sort()

print(biomass_spec[0:9])
print(landings_spec[0:9])

['AFRICAN_POMPANO', 'ALEWIFE', 'AMERICAN_EEL', 'AMERICAN_LOBSTER', 'AMERICAN_SAND_LANCE', 'AMERICAN_SHAD', 'ATLANTIC_ANGEL_SHARK', 'ATLANTIC_ARGENTINE', 'ATLANTIC_BONITO']
['American_plaice', 'Atlantic_butterfish', 'Atlantic_cod', 'Atlantic_croaker', 'Atlantic_halibut', 'Atlantic_herring', 'Atlantic_mackerel', 'Atlantic_redfishes_NS', 'Atlantic_wolffish']


In [183]:
print(biomass_df.columns)
print(landings_df.columns)

biomass_df.drop(['biomass_SE', 'abundance_SE', 'biomass_index_SE', 'abundance_index_units', 'abundance_index_SE'], 
    axis=1, inplace=True)
landings_df.rename(columns={'landings_biomass':'biomass', 'landings_abund':'abundance', 
    'landings_biomass_index':'biomass_index', 'landings_abund_index':'abundance_index'}, inplace=True)
print(biomass_df.columns)
print(landings_df.columns)

Index(['ecosystem', 'species', 'year', 'biomass', 'biomass_SE', 'abundance',
       'abundance_SE', 'biomass_index', 'biomass_index_SE', 'abundance_index',
       'abundance_index_units', 'abundance_index_SE', 'avg_len', 'avg_mass',
       'source', 'agency', 'season'],
      dtype='object')
Index(['ecosystem', 'species', 'year', 'landings_biomass', 'landings_abund',
       'landings_biomass_index', 'landings_abund_index', 'avg_len', 'avg_mass',
       'source', 'agency', 'season'],
      dtype='object')
Index(['ecosystem', 'species', 'year', 'biomass', 'abundance', 'biomass_index',
       'abundance_index', 'avg_len', 'avg_mass', 'source', 'agency', 'season'],
      dtype='object')
Index(['ecosystem', 'species', 'year', 'biomass', 'abundance', 'biomass_index',
       'abundance_index', 'avg_len', 'avg_mass', 'source', 'agency', 'season'],
      dtype='object')


In [184]:
fish_df = pd.concat([biomass_df, landings_df], ignore_index=True)
print(fish_df.tail())
print(fish_df.shape)
print(biomass_df.shape)
print(landings_df.shape)

      ecosystem    species  year  biomass  abundance  biomass_index  \
31017        CB  Skates_NS  2003      NaN        NaN           62.6   
31018        CB  Skates_NS  2005      NaN        NaN           38.7   
31019        CB  Skates_NS  2006      NaN        NaN           20.4   
31020        CB  Skates_NS  2007      NaN        NaN           68.4   
31021        CB  Skates_NS  2008      NaN        NaN          265.7   

       abundance_index  avg_len  avg_mass source agency season  
31017              NaN      NaN       NaN     nd     nd     nd  
31018              NaN      NaN       NaN     nd     nd     nd  
31019              NaN      NaN       NaN     nd     nd     nd  
31020              NaN      NaN       NaN     nd     nd     nd  
31021              NaN      NaN       NaN     nd     nd     nd  
(31022, 12)
(26137, 12)
(4885, 12)


Later modeling is decided to work with recent data only (last 30 years).

In [185]:
max_yr = max(fish_df['year'])
min_yr = max_yr - 30

fish_df_30yr = fish_df[fish_df['year']>= min_yr]
print(fish_df_30yr.shape)

(25776, 12)


Consolidate species list that includes distinct duplicate due to formatting differences (all caps, etc.)

In [186]:
fish_df_30yr['species_lower'] = fish_df_30yr['species'].str.lower()
fish_df_30yr_spec =  list(set(fish_df_30yr['species_lower'].values))
fish_df_30yr_spec.sort()
print(fish_df_30yr_spec[0:9])
len(fish_df_30yr_spec)

['african_pompano', 'alewife', 'alewife_and_blueback_herring', 'alewife_and_river_herring', 'american_butterfish', 'american_eel', 'american_eel_age-1+', 'american_eel_yoy/larvae', 'american_fourspot_flounder']


C:\Users\natha\AppData\Local\Temp\ipykernel_10216\4111078461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fish_df_30yr['species_lower'] = fish_df_30yr['species'].str.lower()


406

In [187]:
species_repl = [species.replace('-','') for species in fish_df_30yr_spec]
species_young = [species for species in fish_df_30yr_spec if ('yoy' in species) or ('y-o-y' in species)]
print(species_young, len(species_young))


['american_eel_yoy/larvae', 'american_sand_lance_yoy', 'anchovy_spp_yoy-est', 'atlantic_herring_yoy-est', 'bay_anchovy_yoy', 'black_sea_bass_yoy', 'blue_catfish_yoy', 'blue_crab_yoy', 'channel_catfish_yoy', 'conger_eel_yoy/larvae', 'gadus_spp._yoy/larvae', 'silver_perch_yoy', 'spot_yoy', 'striped_bass_yoy', 'summer_flounder_yoy', 'weakfish_yoy', 'white_catfish_yoy', 'white_perch_yoy', 'y-o-y_winter_flounder'] 19


Verify "yoy" (young of year) species have adult species data.

In [188]:
species_repl = [species.replace('-est','') for species in species_young]
species_repl = [species.replace('_yoyest','') for species in species_repl]
species_repl = [species.replace('_yoy','') for species in species_repl]
species_repl = [species.replace('y-o-y_','') for species in species_repl]
species_adult = [species.replace('/larvae','') for species in species_repl]
print(species_adult)
len(species_adult)

['american_eel', 'american_sand_lance', 'anchovy_spp', 'atlantic_herring', 'bay_anchovy', 'black_sea_bass', 'blue_catfish', 'blue_crab', 'channel_catfish', 'conger_eel', 'gadus_spp.', 'silver_perch', 'spot', 'striped_bass', 'summer_flounder', 'weakfish', 'white_catfish', 'white_perch', 'winter_flounder']


19

In [190]:
fish_adult = fish_df_30yr[fish_df_30yr['species_lower'].isin(species_adult)]
fish_young = fish_df_30yr[fish_df_30yr['species_lower'].isin(species_young)]
print(fish_adult.shape)
print(fish_young.shape)

(3476, 13)
(332, 13)


In [194]:
print(list(set(fish_adult['species_lower'].values)))
print(len(list(set(fish_adult['species_lower'].values))))
print(list(set(fish_young['species_lower'].values)))
print(len(list(set(fish_young['species_lower'].values))))

['striped_bass', 'channel_catfish', 'weakfish', 'american_sand_lance', 'black_sea_bass', 'american_eel', 'bay_anchovy', 'white_perch', 'blue_crab', 'spot', 'silver_perch', 'white_catfish', 'summer_flounder', 'winter_flounder', 'conger_eel', 'atlantic_herring']
16
['anchovy_spp_yoy-est', 'american_sand_lance_yoy', 'bay_anchovy_yoy', 'silver_perch_yoy', 'y-o-y_winter_flounder', 'striped_bass_yoy', 'white_perch_yoy', 'american_eel_yoy/larvae', 'blue_crab_yoy', 'channel_catfish_yoy', 'conger_eel_yoy/larvae', 'white_catfish_yoy', 'summer_flounder_yoy', 'black_sea_bass_yoy', 'atlantic_herring_yoy-est', 'weakfish_yoy', 'spot_yoy', 'gadus_spp._yoy/larvae', 'blue_catfish_yoy']
19


In [15]:
csv_folder = r"C:\Users\natha\Desktop\bootcamp_repo-1\NW_Atlantic_Fishery_Sustainability\data\interim"
fish_adult.to_csv(fr'{csv_folder}\fish_adult.csv')
fish_young.to_csv(fr'{csv_folder}\fish_young.csv')